In [2]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchmetrics import Precision, Recall
from dataset import ImageDataset
import numpy as np
import datetime
import random
import time
random.seed(2233)
torch.manual_seed(2233)

#After /255 so in loading dataset there are no division by 255 just this normalization
mean = [0.5006, 0.3526, 0.5495]
std = [0.1493, 0.1341, 0.1124]

from albumentations import (
    Compose,
    Resize,
    OneOf,
    RandomBrightness,
    RandomContrast,
    MotionBlur,
    MedianBlur,
    GaussianBlur,
    VerticalFlip,
    HorizontalFlip,
    ShiftScaleRotate,
    Normalize,
)

transform = Compose(
    [
        Normalize(mean=mean, std=std),
        OneOf([RandomBrightness(limit=0.1, p=1), RandomContrast(limit=0.1, p=0.8)]),
        OneOf([MotionBlur(blur_limit=3), MedianBlur(blur_limit=3), GaussianBlur(blur_limit=3),], p=0.7,),
        VerticalFlip(p=0.5),
        HorizontalFlip(p=0.5),
    ]
)

transform_test = Compose(
    [Normalize(mean=mean, std=std)]
)



def objective(trial):
    # Hyperparameters to be tuned
    batch_size = trial.suggest_int('batch_size', 256, 2048)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.8)
    kernel_size = trial.suggest_int('kernel_size', 2, 7)

    # Data loaders
    trainset = ImageDataset(data_path='train_data', transform=transform, reduce=True)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=3)
    
    testset = ImageDataset(data_path='validation_data', transform=transform_test)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Model setup
    model = resnet18()
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(kernel_size, kernel_size), stride=(1, 1), padding=(kernel_size // 2, kernel_size // 2), bias=False)
    num_classes = 4
    model.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(model.fc.in_features, num_classes)
    )
    model = model.to('cuda')

    # Custom model class
    class MyModel(nn.Module):
        def __init__(self, model, learning_rate):
            super(MyModel, self).__init__()
            self.model = model
            self.criterion = nn.CrossEntropyLoss()
            self.optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
            self.metric_precision = Precision(task="multiclass", num_classes=num_classes, average=None).to('cuda')
            self.metric_recall = Recall(task="multiclass", num_classes=num_classes, average=None).to('cuda')
            self.train_loss = []
            self.valid_loss = []

        def forward(self, x):
            return self.model(x)

        def train_one_epoch(self, trainloader):
            self.train()
            for inputs, labels in trainloader:
                inputs, labels = inputs.to('cuda'), labels.to('cuda')
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                self.train_loss.append(loss.item())
            avg_loss = np.mean(self.train_loss)
            self.train_loss.clear()
            return avg_loss

        def evaluate(self, testloader):
            self.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to('cuda'), labels.to('cuda')
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels)
                    self.valid_loss.append(loss.item())
            avg_loss = np.mean(self.valid_loss)
            self.valid_loss.clear()
            return avg_loss

    my_model = MyModel(model=model, learning_rate=learning_rate)
    my_model = my_model.to('cuda')
    early_stop_patience = 15
    num_epochs = 100
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        my_model.train_one_epoch(trainloader)
        val_loss = my_model.evaluate(testloader)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1


        if patience_counter >= early_stop_patience:
            print(f"Early stopping at epoch {epoch} with best validation loss {best_val_loss}")
            break
    return best_val_loss

# Start the optimization
study = optuna.create_study(direction='minimize',
                            storage="sqlite:///db.sqlite3",  
                            study_name="resnet18_finetune_reduced",
                           load_if_exists=True)
start = time.perf_counter()
study.optimize(objective, n_trials=50)
stop = time.perf_counter()
print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")


[I 2024-05-23 00:03:38,844] Using an existing study with name 'resnet18_finetune_reduced' instead of creating a new one.
/tmp/ipykernel_9890/3577430926.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
/tmp/ipykernel_9890/3577430926.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.8)
[I 2024-05-23 00:24:41,590] Trial 1 finished with value: 0.8018940960774656 and parameters: {'batch_size': 1523, 'learning_rate': 5.608525131781936e-06, 'dropout_rate': 0.12120712050207778, 'kernel_size': 7}. Best is trial 1 with value: 0.8018940960774656.
/tmp/ipykernel_98

Early stopping at epoch 52 with best validation loss 0.6903563147481484


/tmp/ipykernel_9890/3577430926.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
/tmp/ipykernel_9890/3577430926.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.8)
[I 2024-05-23 00:57:08,206] Trial 3 finished with value: 0.8255371103684107 and parameters: {'batch_size': 1926, 'learning_rate': 3.559983188392368e-06, 'dropout_rate': 0.5146433953645968, 'kernel_size': 7}. Best is trial 2 with value: 0.6903563147481484.
/tmp/ipykernel_9890/3577430926.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0

Early stopping at epoch 48 with best validation loss 0.7634673002412764


/tmp/ipykernel_9890/3577430926.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
/tmp/ipykernel_9890/3577430926.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.8)
[I 2024-05-23 01:42:31,858] Trial 6 finished with value: 0.6850603044994416 and parameters: {'batch_size': 1492, 'learning_rate': 0.0016821735049478955, 'dropout_rate': 0.6359115886574795, 'kernel_size': 5}. Best is trial 6 with value: 0.6850603044994416.


...

/tmp/ipykernel_9890/3577430926.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
/tmp/ipykernel_9890/3577430926.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.8)
[I 2024-05-23 09:22:32,238] Trial 44 finished with value: 0.6709595084190368 and parameters: {'batch_size': 615, 'learning_rate': 0.001088095449582606, 'dropout_rate': 0.4480204330338669, 'kernel_size': 3}. Best is trial 29 with value: 0.6533146302433724.


Early stopping at epoch 44 with best validation loss 0.6709595084190368


/tmp/ipykernel_9890/3577430926.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
/tmp/ipykernel_9890/3577430926.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.8)
[I 2024-05-23 09:35:14,471] Trial 45 finished with value: 0.6866459177887958 and parameters: {'batch_size': 803, 'learning_rate': 0.00041995542618079504, 'dropout_rate': 0.32503226805827135, 'kernel_size': 4}. Best is trial 29 with value: 0.6533146302433724.


Early stopping at epoch 50 with best validation loss 0.6866459177887958


/tmp/ipykernel_9890/3577430926.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
/tmp/ipykernel_9890/3577430926.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.8)
[W 2024-05-23 09:36:32,947] Trial 46 failed with parameters: {'batch_size': 555, 'learning_rate': 0.006163112767159348, 'dropout_rate': 0.26444603367787456, 'kernel_size': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/adam/miniconda3/envs/cells/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_